In [1]:
# A) Montaje y utilidades
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
import os, json, numpy as np, pandas as pd

# Rutas
P11 = Path("/content/drive/MyDrive/CognitivaAI/p11_alt_backbones")
P18 = Path("/content/drive/MyDrive/CognitivaAI/p18_advanced_stacking")
P18.mkdir(parents=True, exist_ok=True)

VAL_PATH  = P11 / "val_patient_features_backbones.csv"
TEST_PATH = P11 / "test_patient_features_backbones.csv"

# Utilidades métricas
from sklearn.metrics import (
    roc_auc_score, average_precision_score, accuracy_score,
    precision_score, recall_score, f1_score, brier_score_loss
)

def cohort_split(df):
    # OAS1_* vs OAS2_* por patient_id
    is_oas2 = df["patient_id"].astype(str).str.startswith("OAS2_")
    return {"ALL": np.ones(len(df), dtype=bool),
            "OAS1": ~is_oas2.values,
            "OAS2": is_oas2.values}

def eval_bin(y_true, y_prob, thr=0.5):
    y_pred = (y_prob >= thr).astype(int)
    out = dict(
        AUC=float(roc_auc_score(y_true, y_prob)),
        PRAUC=float(average_precision_score(y_true, y_prob)),
        Acc=float(accuracy_score(y_true, y_pred)),
        P=float(precision_score(y_true, y_pred, zero_division=0)),
        R=float(recall_score(y_true, y_pred, zero_division=0)),
        F1=float(f1_score(y_true, y_pred, zero_division=0)),
        thr=float(thr),
        n=int(len(y_true)),
    )
    return out

def best_f1_threshold(y_true, y_prob):
    thresh = np.linspace(0.05, 0.95, 19)
    f1s = [f1_score(y_true, (y_prob>=t).astype(int), zero_division=0) for t in thresh]
    return float(thresh[int(np.argmax(f1s))])


Mounted at /content/drive


In [2]:
# B) Carga y filtrado de columnas (NaN<=0.4)
val = pd.read_csv(VAL_PATH)
test = pd.read_csv(TEST_PATH)
print("VAL:", val.shape, "| TEST:", test.shape)

# Columnas clave
assert {"patient_id","y_true"}.issubset(val.columns)
assert {"patient_id","y_true"}.issubset(test.columns)

# Detectar columnas de features (float) excluyendo metadatos
meta_cols = {"patient_id","y_true"}
feat_cols = [c for c in val.columns if c not in meta_cols]

# Misma intersección en VAL/TEST por seguridad
feat_cols = [c for c in feat_cols if c in test.columns]

# Ratio NaN en VAL
nan_ratio = val[feat_cols].isna().mean().sort_values(ascending=False)
print("\nTop-10 NaN ratio (VAL):\n", nan_ratio.head(10))

keep_cols = [c for c in feat_cols if nan_ratio[c] <= 0.40]
drop_cols = sorted(set(feat_cols)-set(keep_cols))
print(f"\n✅ Mantengo {len(keep_cols)} columnas; ❌ descarto por NaN>0.4: {len(drop_cols)}")

X_val_raw  = val[keep_cols].copy()
X_test_raw = test[keep_cols].copy()
y_val  = val["y_true"].astype(int).values
y_test = test["y_true"].astype(int).values

coh_val  = cohort_split(val)
coh_test = cohort_split(test)


VAL: (69, 58) | TEST: (70, 58)

Top-10 NaN ratio (VAL):
 patient_preds_ensemble_trimmed20    0.855072
patient_preds_ensemble_top7         0.855072
patient_preds_ensemble_p2           0.855072
patient_preds_ensemble_mean         0.855072
patient_preds_trimmed20             0.855072
patient_preds_top7                  0.855072
patient_preds_p2                    0.855072
patient_preds_mean                  0.855072
slices_preds_mean                   0.855072
slices_preds_p2                     0.855072
dtype: float64

✅ Mantengo 36 columnas; ❌ descarto por NaN>0.4: 20


In [3]:
# C) Imputer + flags + StandardScaler (sólo sobre VAL; aplicamos al TEST con los mismos params)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Flags NaN
val_nan_flags  = X_val_raw.isna().astype(int).add_suffix("__isnan")
test_nan_flags = X_test_raw.isna().astype(int).add_suffix("__isnan")

# Imputación mediana
imp = SimpleImputer(strategy="median")
X_val_imp  = pd.DataFrame(imp.fit_transform(X_val_raw), columns=keep_cols, index=X_val_raw.index)
X_test_imp = pd.DataFrame(imp.transform(X_test_raw),  columns=keep_cols, index=X_test_raw.index)

# Estandarización
scaler = StandardScaler()
X_val_std  = pd.DataFrame(scaler.fit_transform(X_val_imp), columns=keep_cols, index=X_val_imp.index)
X_test_std = pd.DataFrame(scaler.transform(X_test_imp),   columns=keep_cols, index=X_test_imp.index)

# Concatenar features + flags
X_val  = pd.concat([X_val_std,  val_nan_flags], axis=1)
X_test = pd.concat([X_test_std, test_nan_flags], axis=1)

print("VAL listo:", X_val.shape, "| TEST listo:", X_test.shape)


VAL listo: (69, 72) | TEST listo: (70, 72)


In [4]:
# D) Base learners + OOF
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm

base_models = {
    "LR_l2": LogisticRegression(max_iter=1000, C=1.0, n_jobs=None),
    "HGB": HistGradientBoostingClassifier(max_depth=None, learning_rate=0.08, max_iter=300, random_state=42),
    "GB": GradientBoostingClassifier(random_state=42),
    "RF": RandomForestClassifier(n_estimators=400, max_depth=None, random_state=42, n_jobs=-1),
    "ET": ExtraTreesClassifier(n_estimators=400, random_state=42, n_jobs=-1)
}

K = 5
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

oof_dict = {name: np.zeros(len(X_val)) for name in base_models}
test_pred_dict = {name: [] for name in base_models}

for name, model in base_models.items():
    print(f"\nOOF para: {name}")
    preds_test_folds = []
    for tr_idx, va_idx in tqdm(skf.split(X_val, y_val), total=K):
        Xtr, Xva = X_val.iloc[tr_idx], X_val.iloc[va_idx]
        ytr, yva = y_val[tr_idx], y_val[va_idx]
        model.fit(Xtr, ytr)
        oof_dict[name][va_idx] = model.predict_proba(Xva)[:,1] if hasattr(model, "predict_proba") else model.decision_function(Xva)
        # Predicción en TEST por fold para promediar
        ptest = model.predict_proba(X_test)[:,1] if hasattr(model, "predict_proba") else model.decision_function(X_test)
        preds_test_folds.append(ptest)
    test_pred_dict[name] = np.mean(preds_test_folds, axis=0)

# Apilamos meta features
meta_val  = np.vstack([oof_dict[k] for k in base_models]).T
meta_test = np.vstack([test_pred_dict[k] for k in base_models]).T
meta_cols = list(base_models.keys())

print("meta_val:", meta_val.shape, "| meta_test:", meta_test.shape)



OOF para: LR_l2


  0%|          | 0/5 [00:00<?, ?it/s]


OOF para: HGB


  0%|          | 0/5 [00:00<?, ?it/s]


OOF para: GB


...


OOF para: RF


  0%|          | 0/5 [00:00<?, ?it/s]


OOF para: ET


  0%|          | 0/5 [00:00<?, ?it/s]

meta_val: (69, 5) | meta_test: (70, 5)


In [5]:
# E) Meta modelos + blending
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier

# 1) Meta LR
meta_lr = LogisticRegression(max_iter=2000)
meta_lr.fit(meta_val, y_val)
val_lr  = meta_lr.predict_proba(meta_val)[:,1]
test_lr = meta_lr.predict_proba(meta_test)[:,1]

# 2) Meta HGB
meta_hgb = HistGradientBoostingClassifier(max_depth=None, learning_rate=0.06, max_iter=600, random_state=42)
meta_hgb.fit(meta_val, y_val)
val_hgb  = meta_hgb.predict_proba(meta_val)[:,1]
test_hgb = meta_hgb.predict_proba(meta_test)[:,1]

# 3) Blending: score = (1-alpha)*HGB + alpha*LR
alphas = np.linspace(0, 0.5, 26)  # favorece HGB pero permite peso LR
best = (-1, None)
for a in alphas:
    v = (1-a)*val_hgb + a*val_lr
    auc = roc_auc_score(y_val, v)
    if auc > best[0]:
        best = (auc, a)
best_auc, best_alpha = best
print(f"\n⭐ Blending óptimo en VAL: alpha={best_alpha:.2f} (LR weight) | AUC={best_auc:.4f}")

val_blend  = (1-best_alpha)*val_hgb  + best_alpha*val_lr
test_blend = (1-best_alpha)*test_hgb + best_alpha*test_lr

# Umbral por F1 en VAL del blend
thr = best_f1_threshold(y_val, val_blend)
print("Umbral F1 (VAL):", thr)

# Métricas consolidada/cohortes
summary = {"VAL":{}, "TEST":{}}
for split, y, p, coh in [
    ("VAL",  y_val,  val_blend,  coh_val),
    ("TEST", y_test, test_blend, coh_test)
]:
    summary[split]["ALL"]  = eval_bin(y, p, thr)
    summary[split]["OAS1"] = eval_bin(y[coh["OAS1"]], p[coh["OAS1"]], thr)
    summary[split]["OAS2"] = eval_bin(y[coh["OAS2"]], p[coh["OAS2"]], thr)

print("\n[VAL]", summary["VAL"])
print("\n[TEST]", summary["TEST"])

# Brier
brier_val  = brier_score_loss(y_val,  val_blend)
brier_test = brier_score_loss(y_test, test_blend)
print(f"\nBrier (VAL)={brier_val:.4f} | (TEST)={brier_test:.4f}")



⭐ Blending óptimo en VAL: alpha=0.02 (LR weight) | AUC=0.9249
Umbral F1 (VAL): 0.44999999999999996

[VAL] {'ALL': {'AUC': 0.9248726655348047, 'PRAUC': 0.9068669804800554, 'Acc': 0.8405797101449275, 'P': 0.7777777777777778, 'R': 0.9032258064516129, 'F1': 0.835820895522388, 'thr': 0.44999999999999996, 'n': 69}, 'OAS1': {'AUC': 0.9740740740740741, 'PRAUC': 0.9706892230576445, 'Acc': 0.9361702127659575, 'P': 0.9047619047619048, 'R': 0.95, 'F1': 0.926829268292683, 'thr': 0.44999999999999996, 'n': 47}, 'OAS2': {'AUC': 0.6983471074380165, 'PRAUC': 0.6533022533022532, 'Acc': 0.6363636363636364, 'P': 0.6, 'R': 0.8181818181818182, 'F1': 0.6923076923076923, 'thr': 0.44999999999999996, 'n': 22}}

[TEST] {'ALL': {'AUC': 0.669407894736842, 'PRAUC': 0.5525173760865196, 'Acc': 0.6571428571428571, 'P': 0.5952380952380952, 'R': 0.78125, 'F1': 0.6756756756756757, 'thr': 0.44999999999999996, 'n': 70}, 'OAS1': {'AUC': 0.6740740740740742, 'PRAUC': 0.5388586381733805, 'Acc': 0.723404255319149, 'P': 0.684210

In [6]:
# F) Importancias (meta HGB) y coeficientes (meta LR) + guardado
import numpy as np

# Importancia de variables en meta (HGB)
try:
    imp = meta_hgb.feature_importances_
    imp_df = pd.DataFrame({"feature": meta_cols, "importance": imp}).sort_values("importance", ascending=False)
    print("\nTop-10 importancias meta (HGB):")
    display(imp_df.head(10))
except Exception as e:
    imp_df = pd.DataFrame({"feature": meta_cols, "importance": np.nan})
    print("No se pudieron extraer importancias HGB:", e)

# Coefs meta (LR)
try:
    coefs = meta_lr.coef_.ravel()
    coef_df = pd.DataFrame({"feature": meta_cols, "coef": coefs}).sort_values("coef", ascending=False)
    print("\nTop-10 coeficientes meta (LR):")
    display(coef_df.head(10))
except Exception as e:
    coef_df = pd.DataFrame({"feature": meta_cols, "coef": np.nan})
    print("No se pudieron extraer coeficientes LR:", e)

# Predicciones por paciente
val_out = pd.DataFrame({
    "patient_id": val["patient_id"],
    "y_true": val["y_true"],
    "y_score": val_blend
})
test_out = pd.DataFrame({
    "patient_id": test["patient_id"],
    "y_true": test["y_true"],
    "y_score": test_blend
})

val_csv  = P18 / "p18_val_patient_preds_ensemble.csv"
test_csv = P18 / "p18_test_patient_preds_ensemble.csv"
val_out.to_csv(val_csv, index=False)
test_out.to_csv(test_csv, index=False)

# Guardar resumen
res = {
    "variant": "p18_advanced_stacking",
    "alpha_lr": float(best_alpha),
    "thr_f1_val": float(thr),
    "VAL": summary["VAL"],
    "TEST": summary["TEST"],
    "brier": {"VAL": float(brier_val), "TEST": float(brier_test)},
    "meta_cols": meta_cols
}
with open(P18 / "p18_ensemble_summary.json", "w") as f:
    json.dump(res, f, indent=2)

imp_df.to_csv(P18 / "p18_meta_importances_hgb.csv", index=False)
coef_df.to_csv(P18 / "p18_meta_coefs_lr.csv", index=False)

print("\n💾 Guardados en:", P18)


No se pudieron extraer importancias HGB: 'HistGradientBoostingClassifier' object has no attribute 'feature_importances_'

Top-10 coeficientes meta (LR):


,feature,coef
2,GB,1.430711
3,RF,0.940762
0,LR_l2,0.567431
4,ET,0.530299
1,HGB,0.252043



💾 Guardados en: /content/drive/MyDrive/CognitivaAI/p18_advanced_stacking


In [7]:
# G) Robustez: permutation importance sobre meta HGB (validación)
from sklearn.inspection import permutation_importance

perm = permutation_importance(
    meta_hgb, meta_val, y_val, n_repeats=20, random_state=42, n_jobs=-1, scoring="roc_auc"
)
perm_df = pd.DataFrame({
    "feature": meta_cols,
    "mean_importance": perm.importances_mean,
    "std_importance": perm.importances_std
}).sort_values("mean_importance", ascending=False)

display(perm_df.head(10))
perm_df.to_csv(P18 / "p18_meta_permutation_importance.csv", index=False)

print("\nChecklist:")
print(" - OOF sin fuga (KFold en VAL) ✅")
print(" - Meta LR + Meta HGB + blending α ✅")
print(" - Umbral F1 de VAL aplicado a TEST ✅")
print(" - Brier + cohortes OAS1/OAS2 ✅")
print(" - Importancias (árbol + permutación) y coeficientes LR ✅")


,feature,mean_importance,std_importance
2,GB,0.292105,0.045660
0,LR_l2,0.134699,0.029492
4,ET,0.060017,0.019825
1,HGB,0.026592,0.009685
3,RF,0.018251,0.010809



Checklist:
 - OOF sin fuga (KFold en VAL) ✅
 - Meta LR + Meta HGB + blending α ✅
 - Umbral F1 de VAL aplicado a TEST ✅
 - Brier + cohortes OAS1/OAS2 ✅
 - Importancias (árbol + permutación) y coeficientes LR ✅
